In [57]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers,optimizers
import numpy as np
import keras

In [2]:
from keras.datasets import cifar10

In [23]:
def PretrainedModel(input_shape, num_of_classes) :
    vgg_16_model = VGG16(weights=None,include_top= False)
    vgg_16_model.summary()
    vgg_16_model.load_weights('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')
    
    print(len(vgg_16_model.layers))
    
    for i in range(19):
        vgg_16_model.layers[i].trainable = False
    
    inputs = Input(shape = (input_shape,input_shape,3), name = "image_input")
    
    output_vgg16_model = vgg_16_model(inputs)
    
    #Add the fully-connected layers 
    x = Flatten(name='flatten')(output_vgg16_model)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(num_of_classes, activation='softmax', name='predictions')(x)

    #Create your own model 
    cifar10_vgg = Model(inputs=inputs, outputs=x)

    #In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
    cifar10_vgg.summary()
    
    print(len(cifar10_vgg.layers))
    
    return cifar10_vgg

    

In [24]:
model = PretrainedModel(32,10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [25]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170491904/170498071 [============================>.] - ETA: 0s

In [28]:
print(x_train[0])

[[[ 59  62  63]
  [ 43  46  45]
  [ 50  48  43]
  ..., 
  [158 132 108]
  [152 125 102]
  [148 124 103]]

 [[ 16  20  20]
  [  0   0   0]
  [ 18   8   0]
  ..., 
  [123  88  55]
  [119  83  50]
  [122  87  57]]

 [[ 25  24  21]
  [ 16   7   0]
  [ 49  27   8]
  ..., 
  [118  84  50]
  [120  84  50]
  [109  73  42]]

 ..., 
 [[208 170  96]
  [201 153  34]
  [198 161  26]
  ..., 
  [160 133  70]
  [ 56  31   7]
  [ 53  34  20]]

 [[180 139  96]
  [173 123  42]
  [186 144  30]
  ..., 
  [184 148  94]
  [ 97  62  34]
  [ 83  53  34]]

 [[177 144 116]
  [168 129  94]
  [179 142  87]
  ..., 
  [216 184 140]
  [151 118  84]
  [123  92  72]]]


In [29]:
print(len(x_train))

50000


In [30]:
print(x_train.shape)

(50000, 32, 32, 3)


In [34]:
np.save("x_train_out.npy",x_train)
np.save("y_train_out.npy",y_train)
np.save("x_test_out.npy",x_test)
np.save("y_test_out.npy",y_test)

In [44]:
x_train = np.load("x_train_out.npy")
y_train = np.load("y_train_out.npy")
x_test = np.load("x_test_out.npy")
y_test = np.load("y_test_out.npy")

In [45]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [46]:
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

In [47]:
print(y_train.shape)

(50000, 1)


In [48]:
print(y_train[0])

[6]


In [49]:
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [50]:
print(y_train.shape)

(50000, 10)


In [52]:
print(y_train[0])

[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]


In [53]:
data_augmentation = True

In [61]:
batch_size = 32
epochs = 25

In [62]:
opt = optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [63]:
model.compile(loss = 'categorical_crossentropy',optimizer=opt,metrics = ['accuracy'])

In [ ]:
if data_augmentation :
    print("-------------Using Data augmentation------------")
     # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    
    datagen.fit(x_train)
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,verbose=1,validation_data=(x_test,y_test))
    
else :
    print("-----Not Using Data augmentation---------------")
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
    
    
    

-------------Using Data augmentation------------
Epoch 1/25
 292/1562 [====>.........................] - ETA: 1309s - loss: 1.3901 - acc: 0.5142

In [ ]:
model.save_weights("cifar_rms_epoch25.h5")